# Analise de Vagas

## Imports

In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import string
from difflib import SequenceMatcher 

## Funções

In [23]:
def getSoup(url):
  payload={}
  headers = {
    'User-Agent': 'Mozilla/5.0'
  }

  html = requests.get(url).content
  soup = BeautifulSoup(html, 'html.parser')
  
  return soup

def similar(str1, str2): 
    return SequenceMatcher(None, str1, str2).ratio() 

def verificaPalavraEmGrupo(palavra, grupoAtual, grupos):
    for palavraChave in grupos[grupoAtual]:
        similaridade = similar(palavra.upper(), palavraChave.upper())
        if (similaridade >= 0.8):
            print(f"palavra: {palavra}; palavraChave: {palavraChave}; similaridade: {similaridade}")
            return True
    return False
    

## Obter profissões com maior número de vagas

In [ ]:
cargoList = list()

soup = getSoup("https://www.vagas.com.br/cargo/")
section = soup.find("section", {"class": "conteudo"})
for a in section.find_all('a', href=True):
  cargoList.append(a['href'])

for cargo in cargoList:
  print(cargo)

A saída de streaming foi truncada nas últimas 5000 linhas.
https://www.vagas.com.br/cargo/coordenador-geral
https://www.vagas.com.br/cargo/coordenador-industrial
https://www.vagas.com.br/cargo/coordenador-juridico
https://www.vagas.com.br/cargo/coordenador-juridico-trabalhista
https://www.vagas.com.br/cargo/coordenador-medico
https://www.vagas.com.br/cargo/coordenador-na-area-civel
https://www.vagas.com.br/cargo/coordenador-nacional-de-vendas
https://www.vagas.com.br/cargo/coordenador-operacional
https://www.vagas.com.br/cargo/coordenador-regional
https://www.vagas.com.br/cargo/coordenador-regional-de-logistica
https://www.vagas.com.br/cargo/coordenador-social
https://www.vagas.com.br/cargo/coordenador-tecnico
https://www.vagas.com.br/cargo/coordenador-tecnico-comercial
https://www.vagas.com.br/cargo/coordenador-tecnico-de-informatica
https://www.vagas.com.br/cargo/coordenador-tecnico-de-planejamento
https://www.vagas.com.br/cargo/coordenador-tecnico-de-projetos
https://www.vagas.com.b

## Navegar por vagas

In [ ]:
listCargos = list()
dictCargo = dict()
for cargo in cargoList:
  try:
    soup = getSoup(cargo)
    link = soup.find('a', {'class': 'button outline'})['href']
    soupCargo = getSoup(link)
    tituloCargo = soupCargo.find('div', {'class': 'titulo-cargo'})
    quantidadeVagas = soupCargo.find('h1')
    if tituloCargo == None or quantidadeVagas == None:
      continue

    dictCargo = {'Cargo': tituloCargo.text.split('Saiba mais')[0].strip(), 'Vagas': quantidadeVagas.text.split(' ')[0].strip(), 'Link': link}
    listCargos.append(dictCargo.copy())

    print(dictCargo)
  except:
    print("Não foi possível obter dados")

{'Cargo': 'Abertura de Contas', 'Vagas': '28', 'Link': 'https://www.vagas.com.br/vagas-de-abertura-de-contas'}
{'Cargo': 'Acabador', 'Vagas': '120', 'Link': 'https://www.vagas.com.br/vagas-de-acabador'}
{'Cargo': 'Acabamento', 'Vagas': '120', 'Link': 'https://www.vagas.com.br/vagas-de-acabamento'}
{'Cargo': 'Acadêmico de Enfermagem', 'Vagas': '23', 'Link': 'https://www.vagas.com.br/vagas-de-acadêmico-de-enfermagem'}
{'Cargo': 'Account', 'Vagas': '26', 'Link': 'https://www.vagas.com.br/vagas-de-account'}
{'Cargo': 'Account Executive', 'Vagas': '1', 'Link': 'https://www.vagas.com.br/vagas-de-account-executive'}
{'Cargo': 'Account Manager', 'Vagas': '5', 'Link': 'https://www.vagas.com.br/vagas-de-account-manager'}
{'Cargo': 'Acompanhante', 'Vagas': '66', 'Link': 'https://www.vagas.com.br/vagas-de-acompanhante'}
{'Cargo': 'Acompanhante Comunitário', 'Vagas': '9', 'Link': 'https://www.vagas.com.br/vagas-de-acompanhante-comunitário'}
{'Cargo': 'Acompanhante Terapêutica', 'Vagas': '1', 'Link'

TypeError: ignored

In [ ]:
df = pd.DataFrame.from_dict(listCargos)
df.to_csv(r'AnaliseDeCargos.csv', index = False, header=True)

## Definir Lista de Profissões

In [ ]:
listProfissoes = []
listProfissoes.append("vagas-de-entregador")

#Navegador Link de Vagas

In [ ]:
pagina = 1
vagaList = list()

for profissao in listProfissoes:
  while True:
    url = f"https://www.vagas.com.br/vagas-de-{profissao}?&q={profissao}&pagina={pagina}&_=1665087383097"
    soup = getSoup(url)
    div = soup.find(id="todasVagas")

    #print(f"Valor de Div = {div}")
    if (div == None):
      break

    for a in div.find_all('a', href=True):
      vagaList.append(a['href'])
    pagina += 1

for vaga in vagaList:
  print(vaga)

## Obter descricao de vaga

In [ ]:
descricaoVagaDict = dict()
descricaoVagaList = list()

for vaga in vagaList:
  soupVaga = getSoup("https://www.vagas.com.br" + vaga)
  tituloVaga = soupVaga.find("h1").text.strip()
  descricaoVaga = soupVaga.find("div", {"class": "job-tab-content job-description__text texto"}).text.strip()

  descricaoVagaDict["Titulo"] = tituloVaga
  descricaoVagaDict["Descricao"] = descricaoVaga

  descricaoVagaList.append(descricaoVagaDict.copy())
  print(descricaoVagaDict)


## Salvar detalhes de vaga em arquivo CSV

In [ ]:
df = pd.DataFrame.from_dict(descricaoVagaList)
df.to_csv(r'AnaliseDeVagas.csv', index = False, header=True)

## Definição das palavras a serem encontradas nas vagas

In [33]:
educacao = ["Educação", "Artes", "Letras", "Pedagogia"]
saude = ["Saúde", "Educação Física", "Enfermagem", "Farmácia", "Fisioterapia", "Licenciatura em Educação Física", "Medicina", "Medicina Veterinária", "Nutrição", "Odontologia", "Psicologia"]
exatasNaturais = ["Exatas", "Naturais", "Ciência da Computação", "Ciências Biológicas", "Licenciatura em Computação", "Matemática", "Química", "Sistemas de Informação"]
humanasComunicacao = ["Humanas", "Comunicação", "Ciências da Religião", "Ciências Sociais", "Comunicação Social", "História", "Moda", "Secretáriado Executivo Bilíngue", "Serviço Social"]
cienciasJuridicas = ["Ciências Jurídicas", "Direito"]
cienciasSociaisAplicadas = ["Ciências Sociais", "Aplicadas", "Administração", "Ciências Contábeis", "Ciências Econômicas", "Tecnologia em Marketing", "Turismo", "Lazer"]
cienciasTecnologicas = ["Ciências Tecnológicas", "Arquitetura", "Urbanismo", "Design", "Engenharia Civil", "Engenharia de Produção", "Engenharia de Telecomunicações", "Engenharia Elétrica", "Engenharia Florestal", "Engenharia Química", "Tecnologia em Processos Industriais"]

grupos = {"educacao": educacao, "saude": saude, "exatasNaturais": exatasNaturais, "humanasComunicacao": humanasComunicacao, "cienciasJuridicas": cienciasJuridicas, "cienciasSociaisAplicadas": cienciasSociaisAplicadas, "cienciasTecnologicas": cienciasTecnologicas}

print(grupos)

{'educacao': ['Educação', 'Artes', 'Letras', 'Pedagogia'], 'saude': ['Saúde', 'Educação Física', 'Enfermagem', 'Farmácia', 'Fisioterapia', 'Licenciatura em Educação Física', 'Medicina', 'Medicina Veterinária', 'Nutrição', 'Odontologia', 'Psicologia'], 'exatasNaturais': ['Exatas', 'Naturais', 'Ciência da Computação', 'Ciências Biológicas', 'Licenciatura em Computação', 'Matemática', 'Química', 'Sistemas de Informação'], 'humanasComunicacao': ['Humanas', 'Comunicação', 'Ciências da Religião', 'Ciências Sociais', 'Comunicação Social', 'História', 'Moda', 'Secretáriado Executivo Bilíngue', 'Serviço Social'], 'cienciasJuridicas': ['Ciências Jurídicas', 'Direito'], 'cienciasSociaisAplicadas': ['Ciências Sociais', 'Aplicadas', 'Administração', 'Ciências Contábeis', 'Ciências Econômicas', 'Tecnologia em Marketing', 'Turismo', 'Lazer'], 'cienciasTecnologicas': ['Ciências Tecnológicas', 'Arquitetura', 'Urbanismo', 'Design', 'Engenharia Civil', 'Engenharia de Produção', 'Engenharia de Telecomunic

## Ler Arquivo CSV do Cargo

In [7]:
administrador = pd.read_csv('./AnaliseDeVagas.csv')



  


    # for grupo in grupos:
    #   for palavraChave in grupo:
    #     similaridade = similar(palavraChave.upper(), palavra.strip().upper())
    #     if (similaridade > 0.75):
    #       print(f"{palavra} - {palavraChave} Similaridade: {similaridade}")

## Definir lista de palavras por vaga

In [8]:
dicionarioPalavraVaga = dict()
listaVagas = list()


descricoes = administrador['Descricao']



for vaga in administrador.to_dict('records'):
    listaPalavrasVaga = list()
    descricao = vaga['Descricao']
    titulo = vaga['Titulo']
    for palavra in descricao.split(' '):
        palavra = palavra.translate(str.maketrans('', '', string.punctuation)).strip()
        listaPalavrasVaga.append(palavra)
    dicionarioPalavraVaga['Titulo'] = titulo
    dicionarioPalavraVaga['Palavras'] = listaPalavrasVaga.copy()
    listaVagas.append(dicionarioPalavraVaga.copy())
    listaPalavrasVaga.clear()


## Definir atributos com base nos grupos para as vagas

In [34]:
listaVagasComAtributos = list()

i = 0
for vaga in listaVagas:
    atributos = {"educacao": 0, "saude": 0, "exatasNaturais": 0, "humanasComunicacao": 0, "cienciasJuridicas": 0, "cienciasSociaisAplicadas": 0, "cienciasTecnologicas": 0}
    for palavra in vaga['Palavras']:
        for grupo in grupos: 
            if (verificaPalavraEmGrupo(palavra, grupo, grupos)):
                atributos[grupo] = 1
    vaga['atributos'] = atributos.copy()
    listaVagasComAtributos.append(vaga.copy())
    i += 1
    if i == 100:
        break

palavra: direto; palavraChave: Direito; similaridade: 0.9230769230769231
palavra: Administração; palavraChave: Administração; similaridade: 1.0
palavra: Administração; palavraChave: Administração; similaridade: 1.0
palavra: administrativo; palavraChave: Administração; similaridade: 0.8148148148148148
palavra: redução; palavraChave: Educação; similaridade: 0.8
palavra: redução; palavraChave: Educação; similaridade: 0.8
palavra: administração; palavraChave: Administração; similaridade: 1.0
palavra: Administrativo; palavraChave: Administração; similaridade: 0.8148148148148148
palavra: administrativo; palavraChave: Administração; similaridade: 0.8148148148148148
palavra: Administração; palavraChave: Administração; similaridade: 1.0
palavra: administração; palavraChave: Administração; similaridade: 1.0
palavra: administrativo; palavraChave: Administração; similaridade: 0.8148148148148148
palavra: odontológica; palavraChave: Odontologia; similaridade: 0.8695652173913043
palavra: Administraçã

In [35]:
for grupo in grupos:
    print(grupo)

educacao
saude
exatasNaturais
humanasComunicacao
cienciasJuridicas
cienciasSociaisAplicadas
cienciasTecnologicas


In [37]:
grupoValores = dict()
for grupo in grupos:
    grupoValores[grupo] = 0
for grupo in grupos:
    for vaga in listaVagasComAtributos:
        valor = vaga['atributos'][grupo]
        grupoValores[grupo] += valor

print(grupoValores)

{'educacao': 16, 'saude': 8, 'exatasNaturais': 6, 'humanasComunicacao': 15, 'cienciasJuridicas': 4, 'cienciasSociaisAplicadas': 47, 'cienciasTecnologicas': 2}
